In [1]:
import sys
import requests
import pprint
from time import sleep

In [2]:
# store global constants
headers = {
   "authorization": "b7d40c7c26294f6eb9bffc672a93384e",
   "content-type": "application/json"
}
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"
upload_endpoint = 'https://api.assemblyai.com/v2/upload'

In [3]:
# make a function to pass the mp3 to the upload endpoint
def read_file(filename):
   with open(filename, 'rb') as _file:
       while True:
           data = _file.read(5242880)
           if not data:
               break
           yield data

In [4]:
from ipywidgets import *
from IPython.display import display, clear_output
from tkinter import Tk, filedialog

In [5]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [6]:
def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename() # List of selected files will be set button's file attribute.
    return b.files # Print the list of files selected.

fileselect = Button(description="File select")

display(fileselect)

file_path = select_files(fileselect)
print(file_path)

# upload our audio file
upload_response = requests.post(
   upload_endpoint,
   headers=headers, data=read_file(file_path)
)
print('Audio file uploaded')

/Users/ashley/Documents/VSCode/metrohacks2021/arctic_tern_test.mp3
Audio file uploaded


In [7]:
# send a request to transcribe the audio file
transcript_request = {'audio_url': upload_response.json()['upload_url']}
transcript_response = requests.post(transcript_endpoint, json=transcript_request, headers=headers)
print('Transcription Requested')
pprint.pprint(transcript_response.json())
# set up polling
polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
filename = os.path.basename(file_path) + '.txt'
# if our status isn’t complete, sleep and then poll again
while polling_response.json()['status'] != 'completed':
   sleep(30)
   polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
   print("File is", polling_response.json()['status'])
with open(filename, 'w') as f:
   f.write(polling_response.json()['text'])
print('Transcript saved to', filename)

Transcription Requested
{'acoustic_model': 'assemblyai_default',
 'audio_duration': None,
 'audio_end_at': None,
 'audio_start_from': None,
 'audio_url': 'https://cdn.assemblyai.com/upload/46260428-064a-41e2-a704-fc6b16333d8e',
 'auto_chapters': False,
 'auto_highlights': False,
 'auto_highlights_result': None,
 'boost_param': None,
 'chapters': None,
 'confidence': None,
 'content_safety': False,
 'content_safety_labels': {},
 'disfluencies': False,
 'dual_channel': None,
 'filter_profanity': False,
 'format_text': True,
 'iab_categories': False,
 'iab_categories_result': {},
 'id': '5c59qqn3c-90a2-4428-bad9-659c267a7bae',
 'language_code': 'en_us',
 'language_model': 'assemblyai_default',
 'punctuate': True,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redact_pii_policies': None,
 'redact_pii_sub': None,
 'sentiment_analysis': False,
 'sentiment_analysis_results': None,
 'speaker_labels': False,
 'speed_boost': False,
 'status': 'queued',
 't

In [9]:
transcription = open(filename,'r')
print(transcription.read())
transcription.close()

Sometimes the unexpected happens. And in the modern world in which the exciting years of childhood seem to be dominated by computer games, a new book arrived at delights the heart and the mind. The Voyage of the Arctic Tent is an epic tale of sailing ships and sailors. It with avarice, sunken treasure and treachery. But good men triumph, and bad men are exposed as base and worthless.
